# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
# To model the database table for the query that gives artist, song title and song length.
# For sessionId = 338 and itemInSession =4
# Partition Key : sessionId
# Clustering Column : itemInSession

In [9]:
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(sessionId int, itemInSession int, artist_name text, length float, song_title text, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO music_library (sessionId, itemInSession, artist_name, length, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
  
        artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId = line
        query_insert = (int(sessionId),int(itemInSession), artist, float(length), song)
        session.execute(query, query_insert)
        

#### Performing a SELECT to verify that the data have been inserted into each table

In [11]:
query =  "SELECT artist_name, song_title, length FROM udacity.music_library WHERE sessionId = 338 AND itemInSession = 4 ;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [12]:
#To model the database table that gives name of artist, song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182
# Partition Key : userId
# Clustering Column: sessionId, itemInSession 

In [13]:
query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + "(userId int, sessionId int, itemInSession int, artist_name text, song_title text, userFirstName text, userLastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO artist_library (userId, sessionId, itemInSession, artist_name, song_title, userFirstName, userLastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
  
        artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId = line
        query_insert = (int(userId), int(sessionId), int(itemInSession), artist, song, firstName, lastName)
        session.execute(query, query_insert)

In [15]:
## SELECT statement to verify the data was entered into the table

query =  "SELECT artist_name, song_title, userfirstname, userlastname FROM udacity.artist_library WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession ALLOW FILTERING"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.userfirstname, row.userlastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [16]:
## To model the database table that gives user name (first and last)
## for song 'All Hands Against His Own'
# Partition key: song_title 
# Clustering Column: userId 

In [17]:
query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(song_title text, userId int, userFirstName text, userLastName text, PRIMARY KEY(song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO song_library (song_title, userId, userFirstName, userLastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
  
        
        artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId = line
        query_insert = (song, int(userId), firstName, lastName)
        session.execute(query, query_insert)

In [19]:
## SELECT statement to verify the data was entered into the table

query =  "SELECT userfirstname, userlastname FROM udacity.song_library WHERE song_title= 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userfirstname, row.userlastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [20]:
query = "DROP TABLE music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [21]:
query = "DROP TABLE artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [22]:
query = "DROP TABLE song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [23]:
session.shutdown()
cluster.shutdown()